In [ ]:
import deep_orderbook as dob
import pandas as pd

In [ ]:
receiver = await dob.recorder.Receiver.create(['ETHBTC'])

In [ ]:
receiver2 = await dob.recorder.Receiver.create(['BNBBTC'])

In [ ]:
b = receiver.depth_managers['ETHBTC'].get_depth_cache().get_bids()
pd.DataFrame(b)

In [ ]:
dir(dob)

In [ ]:
dir(dob.recorder.Receiver)

In [ ]:
receiver